# Graph Neural Network Classification on the PROTEINS Dataset

For the first approach, I'm going to use [Spektral](https://graphneural.network/getting-started/) for Python to build my GCN layer and then perform our classification. 

Spektral is a library for Python for Graph Neural Networks, built on Tensorflow and Keras. 

For the second approach, I'm going to build a GCN from scratch, to experiment with Kipf & Welling's Graph Convolutional Network approach to embedding graphs into vector space. 

In [2]:
# !pip install spektral

In [5]:
# Reading in the PROTEINS dataset
from spektral.datasets import TUDataset

data = TUDataset('PROTEINS')
data

Successfully loaded PROTEINS.


TUDataset(n_graphs=1113)

In [6]:
# Since we want to utilize the Spektral GCN layer, we want to follow the original paper for this method and perform some preprocessing:
from spektral.transforms import GCNFilter

data.apply(GCNFilter())

In [7]:
# Spektral is built on top of Keras, so we can use the Keras functional API to build a model that first embeds,
# then sums the nodes together (global pooling), then classifies the result with a dense softmax layer

# First, let's import the necessary layers:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [9]:
# Now, we can use model subclassing to define our model:

class ProteinsGNN(Model):
  
  def __init__(self, n_hidden, n_labels):
    super().__init__()
    # Define our GCN layer with our n_hidden layers
    self.graph_conv = GCNConv(n_hidden)
    # Define our global pooling layer
    self.pool = GlobalSumPool()
    # Define our dropout layer, initialize dropout freq. to .5 (50%)
    self.dropout = Dropout(0.5)
    # Define our Dense layer, with softmax activation function
    self.dense = Dense(n_labels, 'softmax')

  # Define class method to call model on input
  def call(self, inputs):
    out = self.graph_conv(inputs)
    out = self.dropout(out)
    out = self.pool(out)
    out = self.dense(out)

    return out

In [10]:
# Instantiate our model for training
model = ProteinsGNN(32, data.n_labels)

In [11]:
# Compile model with our optimizer (adam) and loss function
model.compile('adam', 'categorical_crossentropy')

In [14]:
# Here's the trick - we can't just call Keras' fit() method on this model.
# Instead, we have to use Loaders, which Spektral walks us through. Loaders create mini-batches by iterating over the graph
# Since we're using Spektral for an experiment, for our first trial we'll use the recommended loader in the getting started tutorial

# TODO: read up on modes and try other loaders later
from spektral.data import BatchLoader

loader = BatchLoader(data, batch_size=32)

In [15]:
# Now we can train! We don't need to specify a batch size, since our loader is basically a generator
# But we do need to specify the steps_per_epoch parameter

model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

Epoch 1/10
35/35 [==============================] - 3s 28ms/step - loss: 95.9733
Epoch 2/10
35/35 [==============================] - 1s 24ms/step - loss: 14.3282
Epoch 3/10
35/35 [==============================] - 1s 21ms/step - loss: 6.8267
Epoch 4/10
35/35 [==============================] - 1s 23ms/step - loss: 7.8008
Epoch 5/10
35/35 [==============================] - 1s 25ms/step - loss: 6.6979
Epoch 6/10
35/35 [==============================] - 1s 25ms/step - loss: 6.1409
Epoch 7/10
35/35 [==============================] - 1s 25ms/step - loss: 6.2961
Epoch 8/10
35/35 [==============================] - 1s 25ms/step - loss: 6.1376
Epoch 9/10
35/35 [==============================] - 1s 24ms/step - loss: 6.6149
Epoch 10/10
35/35 [==============================] - 1s 26ms/step - loss: 5.6460
